In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sn
import random

In [2]:
Hz = 52

# File names and locations
DATA_DIR = "../data"
CSV_FILES = [os.path.join(DATA_DIR, f) for f 
        in os.listdir(DATA_DIR) 
        if f.endswith('.csv')]

# Column names
DATA_COLS = ["x_accel", "y_accel", "z_accel"] # time series data
TARGET_COL = "target" # the activity
COLS = ["_idx"] + DATA_COLS + [TARGET_COL]

VALID_TARGETS = range(1,8)

In [3]:
def standardize(df):
    """
    Make the mean of data 0 with standard dev of 1
    """
    return (df - df.mean()) / df.std()

In [4]:
dfs = [pd.read_csv(f, names=COLS) for f in CSV_FILES]

In [5]:
train_size = 12
train_dfs = pd.concat(dfs[0:train_size])
val_dfs = pd.concat(dfs[train_size:])

In [6]:
X_train = standardize(train_dfs[DATA_COLS])
X_val = standardize(val_dfs[DATA_COLS])

y_train = train_dfs[TARGET_COL]
y_val = val_dfs[TARGET_COL]

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

data_dim = len(DATA_COLS)
timesteps = 5 * Hz  # 5 seconds
nb_classes = len(VALID_TARGETS)

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(6, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(6, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(6))  # return a single vector of dimension 32
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.fit(X_train, y_train,
          batch_size=30, nb_epoch=5, show_accuracy=True,
          validation_data=(X_val, y_val))

Train on 1459153 samples, validate on 467743 samples
Epoch 1/5


IndexError: indices are out-of-bounds